## IS 590 PR - Final Project 
### Analyzing Olympic Data Set from 1896 to 2016


In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import metrics
import pandas as pd
import dask.dataframe as ddf
import dask.multiprocessing
import warnings
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from pygeodesy.ellipsoidalVincenty import LatLon
from pygeodesy.utily import m2km
from numba import jit
from Latest_Submission import *




warnings.filterwarnings('ignore')



/Users/rj/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


### Reading the Datasets (By Multiprocessing via Dask Dataframes ) -

In [2]:
world_gdp = read_data_df('Data/world_gdp.csv', skiprows = 3)
noc_country = read_data_df('Data/noc_regions.csv')
world_population = read_data_df('Data/world_pop.csv')
olympics = read_data_df('Data/athlete_events.csv') 
olympics_host = read_data_df('Data/olm2.csv')
world_hdi = read_data_df('Data/hdi.csv')




### Shape of the Dataframes
##### get_shape() function returns the rows and columns of Each Dataset 

In [3]:
print(get_shape(olympics))
print(get_shape(world_gdp))
print(get_shape(noc_country))
print(get_shape(world_population))
print(get_shape(olympics_host))
print(get_shape(world_hdi))

(271116, 15)
(219, 61)
(230, 3)
(217, 61)
(51, 7)
(270, 10)


### Assigning Names to the Dataframe

In [4]:
olympics.name = "Olympics"
world_gdp.name = "World GDP"
noc_country.name = "Country Codes"
world_population.name = "World Population"
olympics_host.name = "Olympics Host Countries"
world_hdi.name = "World HDI data"

### Description of the Dataframes

In [5]:
get_info(olympics)
get_info(world_gdp)
get_info(noc_country)
get_info(world_population)
get_info(olympics_host)
get_info(world_hdi)


The Dataframe "Olympics" has 271116 rows and 15 columns
Column Names - 'ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'Medal' 


The Dataframe "World GDP" has 219 rows and 61 columns
Column Names - 'Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016' 


The Dataframe "Country Codes" has 230 rows and 3 columns
Column Names - 'NOC', 'region', 'notes' 


The Dataframe "World Population" has 217 rows and 61 columns
Column Names - 'Country', 'Country Code', 'In

## Data Cleaning
### Missing Values

In [6]:
get_missing_values(olympics)

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [7]:
olympics['Medal'].fillna('No_Medal', inplace=True)

### Validating Datatypes

In [8]:
print(" \n \n{} Dataframe \n \n {} ".format(olympics.name,olympics.dtypes))
print(" \n \n{} Dataframe \n \n {} ".format(world_gdp.name,world_gdp.dtypes))
print(" \n \n{} Dataframe \n \n {} ".format(noc_country.name,noc_country.dtypes))
print(" \n \n{} Dataframe \n \n {} ".format(olympics_host.name,olympics_host.dtypes))
print(" \n \n{} Dataframe \n \n {} ".format(world_hdi.name,world_hdi.dtypes))
print(" {} Dataframe \n \n {} ".format(world_population.name,world_population.dtypes))
world_population[['1998','2000','2016']] = world_population[['1998','2000','2016']].astype(float)

 
 
Olympics Dataframe 
 
 ID          int64
Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
Team       object
NOC        object
Games      object
Year        int64
Season     object
City       object
Sport      object
Event      object
Medal      object
dtype: object 
 
 
World GDP Dataframe 
 
 Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
1960              float64
                   ...   
2012              float64
2013              float64
2014              float64
2015              float64
2016              float64
Length: 61, dtype: object 
 
 
Country Codes Dataframe 
 
 NOC       object
region    object
notes     object
dtype: object 
 
 
Olympics Host Countries Dataframe 
 
 Host City             object
Country               object
Summer (Olympiad)     object
Winter                object
Year                   int64
latitude             float64
longitude            float64
dtyp

### One Hot Encoding of Medals Column

#### Reference -  https://www.datacamp.com/community/tutorials/categorical-data


In [9]:
olympics["Bronze_Medal"] = np.where(olympics["Medal"].str.contains("Bronze"), 1, 0)
olympics["Silver_Medal"] = np.where(olympics["Medal"].str.contains("Silver"), 1, 0)
olympics["Gold_Medal"]   = np.where(olympics["Medal"].str.contains("Gold"), 1, 0)
olympics["No_Medal"]     = np.where(olympics["Medal"].str.contains("No_Medal"), 1, 0)

In [10]:
world = world_gdp.drop(['Indicator Name', 'Indicator Code'], axis = 1)


### Merging Olympic Athletes Data with NOC

In [11]:
olympics_NOC = olympics.merge(noc_country, left_on='NOC' , right_on='NOC' , how ='left' )

### Cleaning Olympics Host data + Extracting years from it

In [12]:
olympics_host['Year'].fillna(-999, inplace=True)
olympics_host['Winter'].fillna("No_Olympics", inplace=True)
olympics_host['Summer (Olympiad)'].fillna("No_Olympics", inplace=True)
olympic_years = list(olympics_host["Year"].unique())
olympic_years = [str(int(element)) for element in olympic_years]



#### Merging World Population Data with Olympic Athletes (after filtering out the years where olympics wasn't held)

In [13]:
@jit
def filter_years(dataframe, years, olympic_years):
    for item in world_population_columns:
        if item not in olympic_years:
            dataframe.drop(item, axis=1, inplace=True)

In [14]:
world_population_columns = list(world_population.columns.values)[4:]

filter_years(world_population,world_population_columns ,olympic_years )
olympics_NOC_population = olympics_NOC.merge(world_population, left_on='NOC' , right_on='Country Code' , how ='left' ).reset_index(drop=True)


#### Merging World GDP Data with Olympic Athletes (after filtering out the years where olympics wasn't held)


In [15]:

world_gdp_columns = list(world_gdp.columns.values)[4:]

filter_years(world_gdp,world_gdp_columns,olympic_years )

olympics_NOC_gdp = olympics_NOC.merge(world_population, left_on='NOC' , right_on='Country Code' , how ='left' ).reset_index(drop=True)


### American Athletes vs World  - Medals Tally in Olympics . 

In [16]:

american_olympians = olympics_NOC.loc[(olympics_NOC.NOC=='USA') & (olympics_NOC.Medal=='Gold') & (olympics_NOC.Season=='Summer')]
american_olympians

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,...,City,Sport,Event,Medal,Bronze_Medal,Silver_Medal,Gold_Medal,No_Medal,region,notes
283,150,Margaret Ives Abbott (-Dunne),F,23.0,NaN,NaN,United States,USA,1900 Summer,1900,...,Paris,Golf,Golf Women's Individual,Gold,0,0,1,0,USA,NaN
609,351,Julius Shareef Abdur-Rahim,M,23.0,202.0,104.0,United States,USA,2000 Summer,2000,...,Sydney,Basketball,Basketball Men's Basketball,Gold,0,0,1,0,USA,NaN
710,404,"Louis Grenville ""Lou"" Abell",M,15.0,NaN,NaN,Vesper Boat Club,USA,1900 Summer,1900,...,Paris,Rowing,Rowing Men's Coxed Eights,Gold,0,0,1,0,USA,NaN
711,404,"Louis Grenville ""Lou"" Abell",M,19.0,NaN,NaN,Vesper Boat Club,USA,1904 Summer,1904,...,St. Louis,Rowing,Rowing Men's Coxed Eights,Gold,0,0,1,0,USA,NaN
739,423,Michael Brent Abernathy,M,22.0,182.0,84.0,United States,USA,2000 Summer,2000,...,Sydney,Baseball,Baseball Men's Baseball,Gold,0,0,1,0,USA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266319,133237,"Timothy R. ""Tim"" Young",M,26.0,172.0,77.0,United States,USA,2000 Summer,2000,...,Sydney,Baseball,Baseball Men's Baseball,Gold,0,0,1,0,USA,NaN
267204,133685,Mariel Leigh Zagunis,F,19.0,173.0,72.0,United States,USA,2004 Summer,2004,...,Athina,Fencing,"Fencing Women's Sabre, Individual",Gold,0,0,1,0,USA,NaN
267205,133685,Mariel Leigh Zagunis,F,23.0,173.0,72.0,United States,USA,2008 Summer,2008,...,Beijing,Fencing,"Fencing Women's Sabre, Individual",Gold,0,0,1,0,USA,NaN
269922,135045,Rbert Zimonyi,M,46.0,170.0,52.0,United States,USA,1964 Summer,1964,...,Tokyo,Rowing,Rowing Men's Coxed Eights,Gold,0,0,1,0,USA,NaN


#### We can see that the Gold medals won by America is 1127 (according to Wikipedia) , but the number os rows returned is 2472 . This is due to medallists in team events being counted as individual medallists. Eg : - Soccer has 11 players , It will be counted as 11 medals instead of 1 .

#### Let's Find out Team Events

In [17]:
gold_medals = olympics_NOC[olympics_NOC.Gold_Medal==1]
gold = gold_medals.groupby(['Event','Year'])['ID'].count()
gold
gold_medals['Event_Frequency'] = gold_medals.groupby(['Event','Year'])['ID'].transform('count')
gold = gold_medals[gold_medals.Event_Frequency > 1]
team_events = gold["Event"].unique()
team_events

array(["Tug-Of-War Men's Tug-Of-War", "Gymnastics Men's Team All-Around",
       "Gymnastics Men's Pommelled Horse", "Handball Women's Handball",
       "Speed Skating Men's 1,500 metres", 'Sailing Mixed 8 metres',
       "Cycling Men's Road Race, Team",
       "Canoeing Men's Kayak Fours, 1,000 metres",
       "Handball Men's Handball", "Football Men's Football",
       "Water Polo Men's Water Polo",
       'Sailing Mixed Two Person Heavyweight Dinghy',
       "Rowing Men's Quadruple Sculls", "Rowing Men's Double Sculls",
       "Rowing Men's Coxed Pairs", "Hockey Men's Hockey",
       "Basketball Men's Basketball", "Nordic Combined Men's Team",
       "Ice Hockey Men's Ice Hockey", "Rowing Men's Coxed Eights",
       "Baseball Men's Baseball", "Athletics Men's 4 x 400 metres Relay",
       "Gymnastics Men's Team All-Around, Free System",
       "Gymnastics Men's Team All-Around, Swedish System",
       "Bobsleigh Men's Two", "Curling Men's Curling",
       "Cycling Men's Team Pursuit

### Medals Tally of USA -

In [18]:
tally = olympics_NOC[(olympics_NOC.No_Medal != 1) & (olympics_NOC.Season == "Summer")]
medal_tally = tally[[ 'NOC','Year', 'Sport', 'Event', 'Medal','Bronze_Medal','Silver_Medal', 'Gold_Medal']]

medal_tally_1 = medal_tally.drop_duplicates(['Medal','Event','Year'])

### Gold Medallists of USA - 


In [19]:
medal_tally_1[(medal_tally_1['Medal']=='Gold') & (medal_tally_1['NOC']=='USA')]

,NOC,Year,Sport,Event,Medal,Bronze_Medal,Silver_Medal,Gold_Medal
283,USA,1900,Golf,Golf Women's Individual,Gold,0,0,1
609,USA,2000,Basketball,Basketball Men's Basketball,Gold,0,0,1
710,USA,1900,Rowing,Rowing Men's Coxed Eights,Gold,0,0,1
711,USA,1904,Rowing,Rowing Men's Coxed Eights,Gold,0,0,1
739,USA,2000,Baseball,Baseball Men's Baseball,Gold,0,0,1
...,...,...,...,...,...,...,...,...
266222,USA,1952,Athletics,Athletics Men's Javelin Throw,Gold,0,0,1
266270,USA,1992,Athletics,Athletics Men's 400 metres Hurdles,Gold,0,0,1
266293,USA,1932,Art Competitions,"Art Competitions Mixed Sculpturing, Statues",Gold,0,0,1
267204,USA,2004,Fencing,"Fencing Women's Sabre, Individual",Gold,0,0,1


### Silver Medallists of USA - 

In [20]:
medal_tally_1[(medal_tally_1['Medal']=='Silver') & (medal_tally_1['NOC']=='USA')]

,NOC,Year,Sport,Event,Medal,Bronze_Medal,Silver_Medal,Gold_Medal
186,USA,2004,Wrestling,"Wrestling Men's Featherweight, Freestyle",Silver,0,1,0
286,USA,2008,Softball,Softball Women's Softball,Silver,0,1,0
311,USA,2004,Taekwondo,Taekwondo Women's Featherweight,Silver,0,1,0
1450,USA,1912,Athletics,Athletics Men's Standing High Jump,Silver,0,1,0
1462,USA,1904,Swimming,Swimming Men's Plunge For Distance,Silver,0,1,0
...,...,...,...,...,...,...,...,...
262882,USA,1972,Athletics,Athletics Men's Shot Put,Silver,0,1,0
263273,USA,1968,Shooting,"Shooting Mixed Small-Bore Rifle, Three Positio...",Silver,0,1,0
263511,USA,2000,Archery,Archery Men's Individual,Silver,0,1,0
263619,USA,1924,Swimming,Swimming Men's 100 metres Backstroke,Silver,0,1,0


### Bronze Medal Tally of USA - 

In [ ]:
medal_tally_1[(medal_tally_1['Medal']=='Bronze') & (medal_tally_1['NOC']=='USA')]

### Visualising Overall Tally with individual Medals (Gold, Silver, Bronze)

In [ ]:
medal_tally_1['Tally_Overall'] = medal_tally_1['Bronze_Medal'].astype(int)+medal_tally_1['Silver_Medal'].astype(int)+medal_tally_1['Gold_Medal'].astype(int)
sns.set()
medal_tally_usa = medal_tally_1[medal_tally_1['NOC']=='USA']
medal_tally_usa=pd.pivot_table(medal_tally_usa, index=['NOC','Year'],values=['Bronze_Medal','Silver_Medal','Gold_Medal','Tally_Overall'],aggfunc=np.sum)
medal_tally_usa.plot()




### USA's Medal Tally by Year (sorted by Medal Tally )

In [ ]:
medal_tally_usa.sort_values('Tally_Overall',ascending=False)



### Medal Tally of All Countries By Year (Sorted by Overall Tally)


In [ ]:
medal_tally_by_year=pd.pivot_table(medal_tally_1, index=['NOC','Year'],values=['Bronze_Medal','Silver_Medal','Gold_Medal','Tally_Overall'],aggfunc=np.sum)

medal_tally_by_year.sort_values('Tally_Overall',ascending=False)

### Medal Tally of All Countries  from 1896 - 2016 (Sorted by Overall Tally)

In [ ]:
medal_tally_overall=pd.pivot_table(medal_tally_1, index=['NOC'],values=['Bronze_Medal','Silver_Medal','Gold_Medal','Tally_Overall'],aggfunc=np.sum)
medal_tally_overall = medal_tally_overall.sort_values('Tally_Overall',ascending=False)
medal_tally_overall

### Pie Chart of Medal Distribution of top 20 Countries

In [ ]:
sns.set()

medal_tally_overall=pd.pivot_table(medal_tally_overall.head(20), index=['NOC'],values=['Tally_Overall'],aggfunc=np.sum).plot(kind='pie',subplots=True,figsize=(50,100),
       autopct='%1.1f%%',textprops={'fontsize': 40})

### Home Advantage
##### Countries actively place lucrative bids to get the hosting rights of every Olympics edition . Apart from Socio-political and economic reasons , we try to explore whether Home advantage impacts a given country's medals tally as well
##### Based on the previous and next olympics' medal tally, we can clearly see the home advantage enjoyed by the host country in the below table.

##### Inspired from - https://www.kaggle.com/chadalee/olympics-data-cleaning-exploration-prediction

### Olympics host Countries 

In [ ]:
olympic_host_noc= olympics_host.merge(noc_country, left_on='Country' , right_on=u'region' , how ='left' )

olympic_host_noc = olympic_host_noc.drop_duplicates(['Country','Year','Summer (Olympiad)'])
olympic_host_noc = olympic_host_noc.drop_duplicates(['Country','Year','Winter'])
olympic_host_noc = olympic_host_noc.drop_duplicates(['Country','Year'] , keep='last')
olympic_host_noc = olympic_host_noc[['Year', 'NOC']]
olympic_host_noc.columns = ['Year_of_Hosting', 'Host_Country_code']
olympic_host_noc

### Medal Tally along with Previous and Next Year Tally
##### Reference - https://stackoverflow.com/questions/34855859/is-there-a-way-in-pandas-to-use-previous-row-value-in-dataframe-apply-when-previ


In [ ]:

medal_tally_by_year.reset_index(inplace=True)
medal_tally_by_year = medal_tally_by_year[['NOC', 'Year','Tally_Overall']]
total_medals = medal_tally_by_year[['NOC','Year','Tally_Overall']]




@jit
def previous_year(tally):
    new_tally = np.empty(tally.shape)
    new_tally[0] =0
    for i in range(1, new_tally.shape[0]):
        new_tally[i] = tally[i-1]
    return new_tally

@jit
def next_year(tally):
    new_tally = np.empty(tally.shape)
    new_tally[tally.shape[0]-1] =0
    for i in range(0, new_tally.shape[0]-1):
        new_tally[i] = tally[i+1]
    return new_tally

total_medals['Previous_Year_Tally'] = previous_year(total_medals['Tally_Overall'])
total_medals['Next_Year_Tally'] = next_year(total_medals['Tally_Overall'])

host_country_tally = olympic_host_noc.merge(total_medals , left_on = [ 'Host_Country_code','Year_of_Hosting'] 
                                           , right_on =['NOC', 'Year'] , how = 'left' )


host = host_country_tally.dropna()
host=host[['Year_of_Hosting', 'Host_Country_code','Previous_Year_Tally', 'Tally_Overall', 'Next_Year_Tally']]
host




### Can we Predict the Overall Medal Tally of a given country ?

##### Research Paper - "Standards on age, height and weight in Olympic running events for men" -  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1859632/?page=2 

##### The Research Paper nails down 6 essential factors when it comes to influencing the medal tally of a given country
##### They are as follows - 

[Medal_Tally.png](attachment:Medal_Tally.png)


##### We try to fit a linear regression line having GDP , Population , GDP per Capita and No of athletes of a given country and see how our Predictor Works. 

##### Inspired By - https://www.kaggle.com/chadalee/olympics-data-cleaning-exploration-prediction

In [ ]:
predictor = medal_tally_by_year

### Merging GDP Data with Year of Participation of the country

In [ ]:
gdp = olympics_NOC_gdp.drop_duplicates(['Country','Year'])
gdp =  gdp.drop(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Games', 'Season', 'City', 'Sport', 'Event', 'Medal', 'Bronze_Medal',
       'Silver_Medal', 'Gold_Medal', 'No_Medal', 'region', 'notes', 'Country',
       'Country Code', 'Indicator Name', 'Indicator Code'],axis=1)
gdp.dropna(inplace=True)
gdp = gdp[gdp.Year>=1960]
pivot_gdp = pd.melt(world, id_vars = ['Country Name', 'Country Code'], var_name = 'Year', value_name = 'GDP') #inspired
pivot_gdp.dropna(inplace=True)

pivot_gdp


### Merging Population Data with Year of Participation of the country

In [ ]:
pop = world_population.drop(world_population.columns[[2, 3]], axis=1)
pivot_pop = pd.melt(pop, id_vars = ['Country', 'Country Code'], var_name = 'Year', value_name = 'Population') #inspired
pivot_pop

### Correlation Matrix of the variables

#### We can see that GDP , Population and Per_Capita_GDP have positive correlation with the medals tally with GDP showing the highest correlation (0.67)

### Merged Frame with Corresponding GDP , Population and GDP Per-capita income with Year of Participation of the country

In [ ]:
gdp_pop = pivot_gdp.merge(pivot_pop,left_on =['Country Code' , 'Year'],right_on = ['Country Code' , 'Year'] , how='left' )
gdp_pop.drop_duplicates(['Country Code', 'Year' ])
gdp_pop.dropna(inplace=True)

gdp_pop = gdp_pop[['Country Name' , 'Country Code' , 'Year' , 'GDP' , 'Population']]
gdp_pop
predictor['Year'] = predictor['Year'].astype('str')
pred = gdp_pop.merge(predictor, left_on =['Country Code', 'Year'] , right_on = ['NOC' , 'Year'] , how ='left' )
predictor['Year'] = predictor['Year'].astype('str')
pred.dropna()
pred['Per_Capita_GDP'] = pred['GDP']/pred['Population']
pred.dropna(inplace=True)
pred
#predictor['Population']= g[(predictor['Year']==g['Year']) & (predictor['NOC']== g['NOC'])]['Team']

In [ ]:
correlation_matrix = pred.corr()
correlation_matrix.style.background_gradient(cmap='seismic')

### Normalise and Classify the models 
#### We need to normalise the input data as there is high variance in them
#### We will use MinMax Scaler to Normalise the data


###  Classifer - 
#### We will use linear regression to predict the medal tally of the country


In [ ]:
#Classifier 

scaler = preprocessing.MinMaxScaler()
A = pred[['GDP', 'Population' , 'Per_Capita_GDP']]
A = scaler.fit_transform(A)
B = pred['Tally_Overall']

model = sm.OLS(B, A)
results_model = model.fit()
print(results_model.summary())

B_predicted = results_model.predict(A)
rmse = np.sqrt(metrics.mean_squared_error(B, B_predicted))
print("Root Mean Square Error of Linear Regression " +  str(rmse))


#### We can see that R-Squared Value of linear Regression is 0.59 (59% accuracy) and RMSE is 15.7 which means the predictor is predicting with an error of 15 medals.
#### Can we Improve this preformance ? 

#### Let's try to predict with Random Forest Regression and see it's performance.

### Random Forest Regression

In [ ]:
reg = RandomForestRegressor(n_estimators=100)
reg.fit(A, B)

print("Random Forest Regression : {:.2f}".format(reg.score(A, B)))
B_predict = reg.predict(A)

rmse = np.sqrt(metrics.mean_squared_error(B, B_predict))
print("Root Mean Square Error Of Random Forest Regressor " + str(rmse) )
plt.scatter(B, B_predict)

#gdp['Population'] = str(list(gdp['Year'].values))

#### We can see that R-Squared Value of Random Forest Regression is 0.94 (94% accuracy) and RMSE is 4.85 which means the predictor is predicting with an error of 5 medals.

#### There is a drastic increase in the accuracy with Random Forest Regression

## HDI  (Human Development Index)

### Let's investigate whether HDI has an impact on Medal Tally. In our dataset , we have HDI Data for 3 Olympics years - 1980 , 2000 and 2008

### Merging HDI data with NOC Data 

In [ ]:
world_hdi.dropna(inplace=True)
pivot_hdi = pd.melt(world_hdi, id_vars = ['HDI'], value_name = 'Human_Dev_Index') #inspired
pivot_hdi.dropna(inplace=True)
pivot_hdi_noc = pivot_hdi.merge(noc_country , left_on ='HDI' , right_on='region' , how='left' ) 

pivot_hdi_noc.drop_duplicates(['variable','region','HDI'],keep='last')

pivot_hdi_noc = pivot_hdi_noc[['NOC', 'variable' , 'Human_Dev_Index' ,'HDI' ]]
pivot_hdi_noc.dropna(inplace=True)
pivot_hdi_noc.columns = ['NOC_HDI','Year_HDI', 'Human_Dev_Index' ,'Country_HDI' ]
pivot_hdi_noc

In [ ]:

pred = pred.merge(pivot_hdi_noc , left_on=['Country Name' ,'Year'] , right_on = ['Country_HDI','Year_HDI'] ,how='left' )
pred.dropna(inplace=True)
pred

#### Correlation Matrix with HDI

In [ ]:
correlation_matrix = pred.corr()
correlation_matrix.style.background_gradient(cmap='seismic')

#### Although not significant , HDI does have a positive correlation with the Medal Tally. 
#### Let's investigate to see if it could help us predict the medal tally better

In [ ]:

MinMax = preprocessing.MinMaxScaler()
A = pred[['GDP', 'Population' , 'Per_Capita_GDP','Human_Dev_Index']]
A = MinMax.fit_transform(A)
B = pred['Tally_Overall']
model = sm.OLS(B, A)
results = model.fit()
print(results.summary())
B_predicted = results.predict(A)
rmse = np.sqrt(metrics.mean_squared_error(B, B_predicted))
print(rmse)
print("\n \nRMSE Of Linear Regressor " + str(rmse) )


####  R-Squared Value of linear Regression is 0.69 (69% accuracy) and RMSE is 13 which means the predictor is predicting with an error of 13 medals.

#### Random Forest Regression - 

In [ ]:

reg = RandomForestRegressor(n_estimators=100)
reg.fit(A, B)

B_predict = reg.predict(A)


rmse = np.sqrt(metrics.mean_squared_error(B, B_predict))
print("\n \nRandom Forest Regression Fit: {:.2f}".format(reg.score(A, B)))
print("\n \nRMSE Of Random Forest Regressor " + str(rmse) )
plt.scatter(B, B_predict)
plt.title("Scatter Plot for Random Forrest Regressor")

#### R-Squared Value of Random Forest Regression is 0.96 (96% accuracy) and RMSE is 3.8 which means the predictor is predicting with an error of 4 medals.

#### Again, Random Forest Regression performs better than Linear Regression

## Comparision of per capita GDP when Japan was the host country vs when Japan wasn't
1. Impact on GDP would vary depending on if the country is a developing country or developed country?
2. Additionally impact would also vary if the country hosts Summer/Winter Olympics

Note : We are not considering the above 2 criterias for our analysis
Result : All the countries show an increase in the GDP/capita over a 3 year period

In [ ]:
gdp_pop = gdp_pop.astype({'Year': 'int64'})
gdp_pop.dtypes
host_country_gdp = olympic_host_noc.merge(gdp_pop , left_on = ['Host_Country_code','Year_of_Hosting'] 
                                           , right_on =['Country Code','Year'] , how = 'left' )
host_country_gdp = host_country_gdp.fillna(0)
host_country_gdp = host_country_gdp[host_country_gdp.Year != 0]
host_country_gdp['Per_Capita_GDP'] = host_country_gdp['GDP']/host_country_gdp['Population']

gdp_pop['Per_Capita_GDP'] = gdp_pop['GDP']/gdp_pop['Population']

#Per Capita GDP when Japan was participating
per_capita_gdp_JPN = gdp_pop.loc[gdp_pop['Country Code'] == 'JPN']
print(per_capita_gdp_JPN.head(100))
#Per Capita GDP when Japan was hosting
per_capita_gdp_JPN_host = host_country_gdp.loc[host_country_gdp['Country Code'] == 'JPN']
print(per_capita_gdp_JPN_host.head(10))

In [ ]:
nohostyear = per_capita_gdp_JPN['Year']
nohostGDP = per_capita_gdp_JPN['Per_Capita_GDP']
hostyear = per_capita_gdp_JPN_host['Year_of_Hosting']
hostGDP = per_capita_gdp_JPN_host['Per_Capita_GDP']

In [ ]:
    index = np.arange(len(nohostyear))
    plt.bar(index, nohostGDP)
    plt.xlabel('Participation Year in Olympics', fontsize=5)
    plt.ylabel('Per Capita GDP', fontsize=5)
    plt.xticks(index, nohostyear, fontsize=5,rotation=10)
    plt.title('Changing trends in Japan Per Capita GDP as a participant')
    plt.show()

In [ ]:
    index = np.arange(len(hostyear))
    plt.bar(index, hostGDP)
    plt.xlabel('Hosting Year in Olympics', fontsize=5)
    plt.ylabel('Per Capita GDP', fontsize=10)
    plt.xticks(index, hostyear, fontsize=5,rotation=10)
    plt.title('Changing trends in Japan Per Capita GDP as a host country')
    plt.show()

### Interesting Stat - 

### How far were the cities that hosted Olympics Each Year from the city that hosted the previous year?

#### (Vectorisation + Numba jit compiler)

##### Ref - https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6

In [ ]:
#interesting Stat - 

#how far were the cities that hosted Olympics Each Year?

@jit
def getroutedistance(host_latitude, host_longitude) :
      latitude = list(host_latitude)
      longitude = list(host_longitude)
      distance = []
      distance.append(0)
      for i in range(0,len(latitude)-1):
           # print(latitude[i], longitude[i],latitude[i+1], longitude[i+1])
            source = LatLon(latitude[i], longitude[i])
            destination = LatLon(latitude[i+1], longitude[i+1]) 
            distance.append(m2km(source.distanceTo(destination)))
            #distance.append(m2km(distance))
            #print(distance)
      return distance


olympics_host['distance(KM)'] = getroutedistance(olympics_host['latitude'],olympics_host['longitude'] )

olympics_host

## Analysing Physiological Factors
### Variation of Height and Weight of Olymians Historically

#### Average Height of the Olympians over the years

In [ ]:
historical = olympics_NOC.dropna(subset=['Height','Weight']).reset_index()
history_height_mean = historical.groupby(['Year'])['Height'].mean()
history_height_mean.plot()

In [ ]:
#### Standard Deviation of the Olympians over the years

In [ ]:
history_height_std = historical.groupby(['Year'])['Height'].std()
history_height_std.plot()

#### Average Weight of the Olympians over the years

In [ ]:
history_weight_mean = historical.groupby(['Year'])['Weight'].mean()
history_weight_mean.plot()


#### Standard Deviation of the Olympians over the years

In [ ]:
history_weight_std = historical.groupby(['Year'])['Weight'].std()
history_weight_std.plot()


## We Analyse the Claims made by a Research Paper based on 1972 Olympics , when it comes to the Height , Weight and Age of the athletes
#### Research Paper - "Standards on age, height and weight in Olympic running events for men"
#### The paper claims that Medalists in 100m Sprint event are Taller and heavier than other participants.
#### Additionally, we try to explore this claim for Age and BMI as well.
#### Overall Participants Average Height ,Height of the Gold Medalist and Height of All Medalists ( Gold , Silver , Bronze ) of 100m Running Event From 1896 -2016


In [ ]:
 sprinter_stats(olympics_NOC , 'Height')

In [ ]:
sprinter_stats(olympics_NOC , 'Height')

In [ ]:
sprinter_stats(olympics_NOC , 'Height').set_index('Year').plot(figsize=(20,8))


In [ ]:
sprinter_stats(olympics_NOC , 'Weight')

In [ ]:
sprinter_stats(olympics_NOC , 'Weight').set_index('Year').plot(figsize=(20,8))

In [ ]:
sprinter_stats(olympics_NOC , 'Age')

In [ ]:
sprinter_stats(olympics_NOC , 'Age').set_index('Year').plot(figsize=(20,8))



### Does history impact olympics? 


#### Comparision of athelete participation over summer and winter olympics
It is known that the participation of athletes varies based on the type of olympics.It can be inferred that summer olympics observes more participation owing to various factors like more participating countries in summer olympics, more events conducted in summer olympics and higher facilities provided for summer olympics




In [ ]:
athletes(olympics_NOC, 'Summer')
athletes(olympics_NOC, 'Winter')

##### We can see a noticible dip in participation at 1936 , 1956and 1980  and No Participation during 1940  , 1944
##### On further exploration , we could see that the following Events Led to drastic change in the participation
##### 1936 Summer Olympics (Berlin, Germany) -  Hitler’s rise to power, less countries were invited , Racism .
##### 1940 and 1944 Summer Olympics (not held due to World War II)
##### 1956 Summer Olympics ( Melbourne, Australia) -  Suez Crisis (Egypt, Iraq, and Lebanon ) , Hungarian revolution (Netherlands, Spain, and Switzerland), Republic of China ( Formosa) . 
##### 1980 Summer Olympics (Moscow, Soviet Union) - Soviet invasion of Afghanistan (USA and its allies boycotted)
##### 1984 Summer Olympics (Los Angeles,  United States ) -Russia Boycotted (Security Reasons)





### Comparision of the number of events in Summer and Winter Olympics
It can be observed that summer olympics has more events in comparision to winter olympics.
Summer olympics observes higher number of events because of factors like :
1. It took place before winter olympics started
2. Geographic & infrastructure requirements required for winter olympics

In [ ]:
events(olympics_NOC_gdp , 'Summer' )
events(olympics_NOC_gdp , 'Winter' )

### Analyzing female participation trend over the years
It can be observed that female particpation increased tremendously over the years. 
Below are certain factors that contribute towards it:
1. Olympics introduced new women events every year
2. Reputation of Olympics impacted countries to encourage women participation

In [ ]:
FemaleTrend(olympics_NOC,'F','Summer')
FemaleTrend(olympics_NOC,'F','Winter')

In [ ]:
# Deleting the columns where Gold Medal is 0
ZeroGold = olympics_NOC[ olympics_NOC['Gold_Medal'] == 0 ].index
 
# Delete these row indexes
olympics_NOC.drop(ZeroGold , inplace=True)

#### Analyzing top 5 countries who won gold medals over the period in Summer olympics

In [ ]:
y = olympics_NOC.groupby(['Season','Sport','region'])['Gold_Medal'].count().reset_index(name="count")
OnlySummer = y[y['Season'] == 'Summer'] 
idx = OnlySummer.groupby(['Sport'])['count'].transform(max) == OnlySummer['count']
OnlySummer[idx].sort_values(by=['count'],ascending=False)
a=OnlySummer[idx].groupby(['region'])['count'].sum().reset_index(name="Summer Gold Medal")
d=a.nlargest(5,'Summer Gold Medal')
d

#### Analyzing top 5 countries who won gold medals over the period in Winter olympics
It can be observed that Nordic countries perform better in Winter olympics than Summer olympics owing to geographic advantage,
better winter events training facilities

In [ ]:
OnlyWinter = y[y['Season'] == 'Winter'] 
indx1 = OnlyWinter.groupby(['Sport'])['count'].transform(max) == OnlyWinter['count']
OnlyWinter[indx1].sort_values(by=['count'],ascending=False)
b=OnlyWinter[indx1].groupby(['region'])['count'].sum().reset_index(name="Winter Gold Medal")
c=b.nlargest(5,'Winter Gold Medal')
c

#### Calculating top 3 PERFORMING events for USA,RUSSIA AND UK IN SUMMER OLYMPICS 

In [ ]:
a1=OnlySummer[idx].groupby(['region','Sport'])['count'].sum().reset_index(name="Summer Gold Medal")
z=a1.groupby(['region','Sport'], sort=False)['Summer Gold Medal'].max().reset_index(name="Summer Count of Gold Medal")

k=z[z['region'] == 'USA']
l=k.nlargest(5,'Summer Count of Gold Medal')
print(l)

k1=z[z['region'] == 'Russia']
l1=k1.nlargest(5,'Summer Count of Gold Medal')
print(l1)

k2=z[z['region'] == 'UK']
l2=k2.nlargest(5,'Summer Count of Gold Medal')
print(l2)


#### Calculating top 3 PERFORMING events for USA,GERMANY AND NORWAY IN WINTER OLYMPICS

In [ ]:
c1=OnlyWinter[indx1].groupby(['region','Sport'])['count'].sum().reset_index(name="Winter Gold Medal")
z=c1.groupby(['region','Sport'], sort=False)['Winter Gold Medal'].max().reset_index(name="Winter Count of Gold Medal")

m=z[z['region'] == 'USA']
n=m.nlargest(5,'Winter Count of Gold Medal')
print(n)

m1=z[z['region'] == 'Germany']
n1=m1.nlargest(5,'Winter Count of Gold Medal')
print(n1)

m2=z[z['region'] == 'Norway']
n2=m2.nlargest(5,'Winter Count of Gold Medal')
print(n2)

## References

### Datasets 

##### Olympics Dataset , NOC -  https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results

##### GDP Dataset -  https://datahub.io/core/gdp

##### Population Dataset - https://vizhub.com/celtic660/datasets/world_pop

##### Human Development Index - https://data.world/brianray/gapminder-hdi-human-developmen

#### Olympics Host - https://www.kaggle.com/pavanraj159/olympic-host



### Books and Research Papers 

##### 1. Jake VanderPlas. 2016. <em>Python Data Science Handbook: Essential Tools for Working with Data</em> (1st ed.). O'Reilly Media, Inc.
##### 2.  McKinney: "Python for Data Analysis", 2nd Edition, by Wes McKinney, ©2017.
##### 3. “Going for Gold Medals: Factors affecting Olympic Performance” -
“https://www.researchgate.net/publication/278585677_Going_for_Gold_Medals_Factors_affecting_Olympic_Performance
##### 4."Standards on age, height and weight in Olympic running events for men" -  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1859632/?page=2

### Online References 

##### 1. https://en.wikipedia.org/wiki/Olympic_Games
##### 2. https://stackoverflow.com/  
##### 3. https://www.olympic.org/
##### 4. https://towardsdatascience.com/understanding-the-need-for-optimization-when-using-pandas-8ce23b83330c
##### 5. https://www.kaggle.com/chadalee/olympics-data-cleaning-exploration-prediction
##### 6. https://www.kaggle.com/pavanraj159/olympics-history-1896-2016
##### 7. https://www.kaggle.com/arunsankar/key-insights-from-olympic-history-data
##### 8. https://en.wikipedia.org/wiki/List_of_Olympic_Games_scandals_and_controversies
